# Smoothing trajectories

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=1-tutorials/10-smoothing-trajectories.ipynb)
[![IPYNB](https://img.shields.io/badge/view-ipynb-hotpink)](https://github.com/anitagraser/movingpandas-examples/blob/main/1-tutorials/10-smoothing-trajectories.ipynb)
[![HTML](https://img.shields.io/badge/view-html-green)](https://anitagraser.github.io/movingpandas-website/1-tutorials/10-smoothing-trajectories.html)

To smooth trajectories, we can use a Kalman filter. The implemented KalmanSmootherCV is based on the assumption of a nearly-constant velocity (CV) model. To use KalmanSmootherCV, the optional dependency `StoneSoup` needs to be installed.

[Documentation](https://movingpandas.readthedocs.io/en/master/trajectorysmoother.html)

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import movingpandas as mpd

import warnings
warnings.filterwarnings('ignore')

mpd.show_versions()

In [ ]:
gdf = read_file('../data/geolife_small.gpkg')
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id', t='t')

In [ ]:
split = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=15))

In [ ]:
plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
split.plot(column='trajectory_id', **plot_defaults)

## KalmanSmootherCV

This smoother operates on the assumption of a nearly-constant velocity (CV) model. The `process_noise_std` and `measurement_noise_std` parameters can be used to tune the smoother:

* `process_noise_std` governs the uncertainty associated with the adherence of the new (smooth) trajectories to the CV model assumption; higher values relax the assumption, therefore leading to less-smooth trajectories, and vice-versa.
* `measurement_noise_std` controls the assumed error in the original trajectories; higher values dictate that the original trajectories are expected to be noisier (and therefore, less reliable), thus leading to smoother trajectories, and vice-versa.

Try tuning these parameters and observe the resulting trajectories:

In [ ]:
help(mpd.KalmanSmootherCV)

In [ ]:
smooth = mpd.trajectory_smoother.KalmanSmootherCV(split).smooth(process_noise_std=0.1, measurement_noise_std=10)
print(smooth)

In [ ]:
hvplot_defaults = {'tiles':'CartoLight', 'frame_height':320, 'frame_width':320, 'cmap':'Viridis', 'colorbar':True}
kwargs = {**hvplot_defaults, 'line_width':4}
(split.hvplot(title='Original Trajectories', **kwargs) + 
 smooth.hvplot(title='Smooth Trajectories', **kwargs))

In [ ]:
kwargs = {**hvplot_defaults, 'c':'speed', 'line_width':7, 'clim':(0,20)}
(split.trajectories[2].hvplot(title='Original Trajectories', **kwargs) + 
 smooth.trajectories[2].hvplot(title='Smooth Trajectories', **kwargs))